<a href="https://colab.research.google.com/github/Rajsoni03/Covid-X-ray-Dataset/blob/main/Data_Collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#importing Libraries

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from imutils import paths

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import shutil
import cv2
import os

#Create Dirs

In [2]:
os.mkdir('/content/data')
os.mkdir('/content/data/train')
os.mkdir('/content/data/val')
os.mkdir('/content/data/test')


os.mkdir('/content/data/train/covid')
os.mkdir('/content/data/train/normal')
os.mkdir('/content/data/train/pneumonia')

os.mkdir('/content/data/val/covid')
os.mkdir('/content/data/val/normal')
os.mkdir('/content/data/val/pneumonia')

os.mkdir('/content/data/test/covid')
os.mkdir('/content/data/test/normal')
os.mkdir('/content/data/test/pneumonia')


In [3]:
main_dir = "/content/data"

def random_split(images, base_dir, class_name):
  length = len(images)
  images = images
  np.random.shuffle(images)
  
  no_of_train_end  = int(length * 0.8)
  no_of_val_end  = int(length * 0.9)

  print('Class:', class_name,'\nTrain Imgs:', no_of_train_end,'\nVal Imgs:',length-no_of_val_end,'\nTest Imgs:',  no_of_val_end-no_of_train_end, end='\n\n')

  for img in images[:no_of_train_end]:
    # print(os.path.join(base_dir, img), os.path.join(os.path.join(os.path.join(main_dir, 'train'), class_name), img))
    shutil.copy(os.path.join(base_dir, img), os.path.join(os.path.join(os.path.join(main_dir, 'train'), class_name), img))
  for img in images[no_of_val_end:]:
    # print(os.path.join(base_dir, img), os.path.join(os.path.join(os.path.join(main_dir, 'val'), class_name), img))
    shutil.copy(os.path.join(base_dir, img), os.path.join(os.path.join(os.path.join(main_dir,  'val'), class_name), img))
  for img in images[no_of_train_end: no_of_val_end]:
    # print(os.path.join(base_dir, img), os.path.join(os.path.join(os.path.join(main_dir, 'test'), class_name), img))
    shutil.copy(os.path.join(base_dir, img), os.path.join(os.path.join(os.path.join(main_dir, 'test'), class_name), img))

#Data Source 1

In [4]:
# clone Data
!git clone https://github.com/agchung/Actualmed-COVID-chestxray-dataset.git

Cloning into 'Actualmed-COVID-chestxray-dataset'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 422 (delta 1), reused 6 (delta 1), pack-reused 412
Receiving objects: 100% (422/422), 1.56 GiB | 36.59 MiB/s, done.
Resolving deltas: 100% (16/16), done.
Checking out files: 100% (240/240), done.


In [5]:
df = pd.read_csv('/content/Actualmed-COVID-chestxray-dataset/metadata.csv')
df.head()

,patientid,offset,sex,age,finding,survival,temperature,pO2 saturation,view,modality,imagename,artifacts/distortion,notes
0,ANON196,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PA,X-ray,CR.1.2.840.113564.1722810170.20200313090023421...,NaN,"Inconclusive, date taken (YYYYMMDD): 20200313"
1,ANON196,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PA,X-ray,CR.1.2.840.113564.1722810170.20200313100318375...,NaN,"Inconclusive, date taken (YYYYMMDD): 20200313"
2,ANON56,NaN,NaN,NaN,No finding,NaN,NaN,NaN,PA,X-ray,CR.1.2.840.113564.1722810170.20200317090830828...,NaN,Negative (covid-19 viral infection is excluded...
3,ANON56,NaN,NaN,NaN,No finding,NaN,NaN,NaN,PA,X-ray,CR.1.2.840.113564.1722810170.20200317104341875...,NaN,Negative (covid-19 viral infection is excluded...
4,ANON196,NaN,NaN,NaN,COVID-19,NaN,NaN,NaN,PA,X-ray,CR.1.2.840.113564.1722810170.20200318082923328...,NaN,", date taken (YYYYMMDD): 20200318"


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 238 entries, 0 to 237
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   patientid             238 non-null    object 
 1   offset                0 non-null      float64
 2   sex                   0 non-null      float64
 3   age                   0 non-null      float64
 4   finding               185 non-null    object 
 5   survival              0 non-null      float64
 6   temperature           0 non-null      float64
 7   pO2 saturation        0 non-null      float64
 8   view                  238 non-null    object 
 9   modality              238 non-null    object 
 10  imagename             238 non-null    object 
 11  artifacts/distortion  0 non-null      float64
 12  notes                 238 non-null    object 
dtypes: float64(7), object(6)
memory usage: 24.3+ KB


In [7]:
df['finding'].unique()

array([nan, 'No finding', 'COVID-19'], dtype=object)

In [8]:
img_src_dir = "/content/Actualmed-COVID-chestxray-dataset/images"

covid_imgs = []
normal_imgs = []

for find, img in np.array(df[['finding', 'imagename']]):
  if find=='COVID-19':
    covid_imgs.append(img)
  if find=='No finding':
    normal_imgs.append(img)

# copy images to covid and normal dir
random_split(covid_imgs, img_src_dir, 'covid')
random_split(normal_imgs, img_src_dir, 'normal')


Class: covid 
Train Imgs: 46 
Val Imgs: 6 
Test Imgs: 6

Class: normal 
Train Imgs: 101 
Val Imgs: 13 
Test Imgs: 13



In [ ]:
print('Covid', len(os.listdir('/content/data/train/covid'))+len(os.listdir('/content/data/test/covid'))+len(os.listdir('/content/data/val/covid')))
print('Normal', len(os.listdir('/content/data/train/normal'))+len(os.listdir('/content/data/test/normal'))+len(os.listdir('/content/data/val/normal')))
print('Pneumonoia', len(os.listdir('/content/data/train/pneumonia'))+len(os.listdir('/content/data/test/pneumonia'))+len(os.listdir('/content/data/val/pneumonia')))

#Data Source 2

In [9]:
!git clone https://github.com/agchung/Figure1-COVID-chestxray-dataset.git

Cloning into 'Figure1-COVID-chestxray-dataset'...
remote: Enumerating objects: 112, done.
remote: Counting objects: 100% (112/112), done.
remote: Compressing objects: 100% (97/97), done.
remote: Total 112 (delta 28), reused 95 (delta 14), pack-reused 0
Receiving objects: 100% (112/112), 14.13 MiB | 35.37 MiB/s, done.
Resolving deltas: 100% (28/28), done.


In [10]:
df = pd.read_csv('/content/Figure1-COVID-chestxray-dataset/metadata.csv', encoding='unicode_escape')
df.head()

,patientid,offset,sex,age,finding,survival,temperature,pO2 saturation,view,modality,artifacts/distortion,notes
0,COVID-00001,13.0,M,33,COVID-19,NaN,NaN,58,AP erect,X-ray,NaN,O2 saturation was initially 58% on room air an...
1,COVID-00002,NaN,M,50+,NaN,NaN,NaN,NaN,AP erect,X-ray,NaN,"50+ male patient, asthmatic, returning from a ..."
2,COVID-00003a,8.0,M,28,NaN,NaN,39.1,90,AP erect,X-ray,NaN,"28M previously fit and well, not on any regula..."
3,COVID-00003b,8.0,M,28,NaN,NaN,39.1,90,AP erect,X-ray,NaN,Post-intubation in the critical care setting
4,COVID-00004,2.0,M,42,COVID-19,NaN,NaN,91-92,PA,X-ray,NaN,42 year old male patient presented to ED with ...


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   patientid             55 non-null     object 
 1   offset                20 non-null     float64
 2   sex                   22 non-null     object 
 3   age                   29 non-null     object 
 4   finding               40 non-null     object 
 5   survival              0 non-null      float64
 6   temperature           4 non-null      object 
 7   pO2 saturation        12 non-null     object 
 8   view                  18 non-null     object 
 9   modality              55 non-null     object 
 10  artifacts/distortion  7 non-null      object 
 11  notes                 48 non-null     object 
dtypes: float64(2), object(10)
memory usage: 5.3+ KB


In [12]:
# add column of Image Names with file extension
img_list = os.listdir("/content/Figure1-COVID-chestxray-dataset/images")
string = ''
for i in img_list:
  string+=i

img_name = []
for i in df['patientid']:
  img_name.append(string[string.find(i):string.find(i)+len(i)+4])

df['img_name'] = img_name  

In [13]:
df.head()

,patientid,offset,sex,age,finding,survival,temperature,pO2 saturation,view,modality,artifacts/distortion,notes,img_name
0,COVID-00001,13.0,M,33,COVID-19,NaN,NaN,58,AP erect,X-ray,NaN,O2 saturation was initially 58% on room air an...,COVID-00001.jpg
1,COVID-00002,NaN,M,50+,NaN,NaN,NaN,NaN,AP erect,X-ray,NaN,"50+ male patient, asthmatic, returning from a ...",COVID-00002.jpg
2,COVID-00003a,8.0,M,28,NaN,NaN,39.1,90,AP erect,X-ray,NaN,"28M previously fit and well, not on any regula...",COVID-00003a.jpg
3,COVID-00003b,8.0,M,28,NaN,NaN,39.1,90,AP erect,X-ray,NaN,Post-intubation in the critical care setting,COVID-00003b.jpg
4,COVID-00004,2.0,M,42,COVID-19,NaN,NaN,91-92,PA,X-ray,NaN,42 year old male patient presented to ED with ...,COVID-00004.jpg


In [14]:
df['finding'].unique()

array(['COVID-19', nan, 'No finding', 'Pneumonia'], dtype=object)

In [15]:
img_src_dir = "/content/Figure1-COVID-chestxray-dataset/images"

covid_imgs = []
normal_imgs = []
pneumonia_imgs = []

for find, img in np.array(df[['finding', 'img_name']]):
  if find=='COVID-19':
    covid_imgs.append(img)
  if find=='No finding':
    normal_imgs.append(img)
  if find=='Pneumonia':
    pneumonia_imgs.append(img)

# copy images to covid and normal dir
random_split(covid_imgs, img_src_dir, 'covid')
random_split(normal_imgs, img_src_dir, 'normal')
random_split(pneumonia_imgs, img_src_dir, 'pneumonia')


Class: covid 
Train Imgs: 28 
Val Imgs: 4 
Test Imgs: 3

Class: normal 
Train Imgs: 2 
Val Imgs: 1 
Test Imgs: 0

Class: pneumonia 
Train Imgs: 1 
Val Imgs: 1 
Test Imgs: 0



In [16]:
print('Covid', len(os.listdir('/content/data/train/covid'))+len(os.listdir('/content/data/test/covid'))+len(os.listdir('/content/data/val/covid')))
print('Normal', len(os.listdir('/content/data/train/normal'))+len(os.listdir('/content/data/test/normal'))+len(os.listdir('/content/data/val/normal')))
print('Pneumonoia', len(os.listdir('/content/data/train/pneumonia'))+len(os.listdir('/content/data/test/pneumonia'))+len(os.listdir('/content/data/val/pneumonia')))


Covid 93
Normal 130
Pneumonoia 2


#Data Source 3

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [24]:
# os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive/AI Colab/Kaggle'
!kaggle datasets download -d tawsifurrahman/covid19-radiography-database
# !kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

 99% 1.14G/1.15G [00:13<00:00, 93.8MB/s]
100% 1.15G/1.15G [00:13<00:00, 91.2MB/s]


In [26]:
#unzipping the zip files and deleting the zip files
!unzip /content/covid19-radiography-database.zip  && /content/covid19-radiography-database.zip

Archive:  /content/covid19-radiography-database.zip
  inflating: COVID-19 Radiography Database/COVID-19.metadata.xlsx  
  inflating: COVID-19 Radiography Database/COVID-19/COVID-19 (1).png  
  inflating: COVID-19 Radiography Database/COVID-19/COVID-19 (10).png  
  inflating: COVID-19 Radiography Database/COVID-19/COVID-19 (100).png  
  inflating: COVID-19 Radiography Database/COVID-19/COVID-19 (101).png  
  inflating: COVID-19 Radiography Database/COVID-19/COVID-19 (102).png  
  inflating: COVID-19 Radiography Database/COVID-19/COVID-19 (103).png  
  inflating: COVID-19 Radiography Database/COVID-19/COVID-19 (104).png  
  inflating: COVID-19 Radiography Database/COVID-19/COVID-19 (105).png  
  inflating: COVID-19 Radiography Database/COVID-19/COVID-19 (106).png  
  inflating: COVID-19 Radiography Database/COVID-19/COVID-19 (107).png  
  inflating: COVID-19 Radiography Database/COVID-19/COVID-19 (108).png  
  inflating: COVID-19 Radiography Database/COVID-19/COVID-19 (109).png  
  infla

In [29]:
print('Covid:',len(os.listdir('/content/COVID-19 Radiography Database/COVID-19')))
print('Normal:',len(os.listdir('/content/COVID-19 Radiography Database/NORMAL')))
print('Pneumonia:',len(os.listdir('/content/COVID-19 Radiography Database/Viral Pneumonia')))

Covid: 219
Normal: 1341
Pneumonia: 1345


In [30]:
covid_img_src_dir = '/content/COVID-19 Radiography Database/COVID-19'
normal_img_src_dir = '/content/COVID-19 Radiography Database/NORMAL'
pneumonia_img_src_dir = '/content/COVID-19 Radiography Database//Viral Pneumonia'

covid_imgs = os.listdir(covid_img_src_dir)
normal_imgs = os.listdir(normal_img_src_dir)
pneumonia_imgs = os.listdir(pneumonia_img_src_dir)

# copy images to covid and normal dir
random_split(covid_imgs, covid_img_src_dir, 'covid')
random_split(normal_imgs, normal_img_src_dir, 'normal')
random_split(pneumonia_imgs, pneumonia_img_src_dir, 'pneumonia')


Class: covid 
Train Imgs: 175 
Val Imgs: 22 
Test Imgs: 22

Class: normal 
Train Imgs: 1072 
Val Imgs: 135 
Test Imgs: 134

Class: pneumonia 
Train Imgs: 1076 
Val Imgs: 135 
Test Imgs: 134



In [32]:
print('Covid', len(os.listdir('/content/data/train/covid'))+len(os.listdir('/content/data/test/covid'))+len(os.listdir('/content/data/val/covid')))
print('Normal', len(os.listdir('/content/data/train/normal'))+len(os.listdir('/content/data/test/normal'))+len(os.listdir('/content/data/val/normal')))
print('Pneumonoia', len(os.listdir('/content/data/train/pneumonia'))+len(os.listdir('/content/data/test/pneumonia'))+len(os.listdir('/content/data/val/pneumonia')))


Covid 312
Normal 1471
Pneumonoia 1347


#Exporting Data

In [63]:
shutil.copytree('/content/data', '/content/drive/MyDrive/AI Colab/Final Covid Diagnosis/data', copy_function = shutil.copy)

'/content/drive/MyDrive/AI Colab/Final Covid Diagnosis/data'